In [1]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Download the dataset
!wget https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip

--2023-09-24 22:26:30--  https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.114.207, 172.217.214.207, 108.177.111.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.114.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M   153MB/s    in 0.4s    

2023-09-24 22:26:30 (153 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [3]:
# Extract the archive
zip_ref = zipfile.ZipFile("./cats_and_dogs_filtered.zip", 'r')
zip_ref.extractall("tmp/")
zip_ref.close()

# Assign training and validation set directories
base_dir = 'tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')


* `rotation_range` is a value in degrees (0–180) within which to randomly rotate pictures.
* `width_shift` and `height_shift` are ranges (as a fraction of total width or height) within which to randomly translate pictures vertically or horizontally.
* `shear_range` is for randomly applying shearing transformations.
* `zoom_range` is for randomly zooming inside pictures.
* `horizontal_flip` is for randomly flipping half of the images horizontally. This is relevant when there are no assumptions of horizontal assymmetry (e.g. real-world pictures).
* `fill_mode` is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift.


Run the next cells to see the impact on the results. The code is similar to the baseline but the definition of `train_datagen` has been updated to use the parameters described above.

In [4]:
# This code has changed. Now instead of the ImageGenerator just rescaling
# the image, we also rotate and do other operations
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [5]:
model_1 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters = 16, kernel_size = (3,3), activation = "relu", input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = "relu"),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")
    ]
)

model_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])


class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_accuracy') is not None and logs.get('val_accuracy') > 0.70:
            print("\nReached 70% val accuracy so cancelling training!")
            self.model.stop_training = True

In [6]:
history_1 = model_1.fit(train_generator,
                        epochs = 20,
                        validation_data=validation_generator,
                        verbose = 1,
                        callbacks = [myCallback()])

Epoch 1/20
100/100 [==============================] - 27s 148ms/step - loss: 0.7019 - accuracy: 0.5110 - val_loss: 0.6913 - val_accuracy: 0.5080
Epoch 2/20
100/100 [==============================] - 15s 149ms/step - loss: 0.6916 - accuracy: 0.5350 - val_loss: 0.6867 - val_accuracy: 0.5670
Epoch 3/20
100/100 [==============================] - 15s 147ms/step - loss: 0.6864 - accuracy: 0.5640 - val_loss: 0.6699 - val_accuracy: 0.6280
Epoch 4/20
100/100 [==============================] - 15s 150ms/step - loss: 0.6860 - accuracy: 0.5420 - val_loss: 0.6835 - val_accuracy: 0.5840
Epoch 5/20
100/100 [==============================] - 16s 160ms/step - loss: 0.6806 - accuracy: 0.5670 - val_loss: 0.6672 - val_accuracy: 0.5990
Epoch 6/20
100/100 [==============================] - 16s 156ms/step - loss: 0.6773 - accuracy: 0.5710 - val_loss: 0.6823 - val_accuracy: 0.5640
Epoch 7/20
100/100 [==============================] - 15s 148ms/step - loss: 0.6781 - accuracy: 0.5800 - val_loss: 0.6541 - val_ac

In [7]:
# More Epoch we need to prevent underfitting